[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)

# SEC EDGAR API in Python -- Free, No API Key

Use **edgartools** to access SEC EDGAR data in Python -- completely free, no API key or paid subscription required. Look up any company, search filings, extract financial statements, track insider trades, and analyze institutional holdings.

**What you'll learn:**
- Look up any company by ticker, CIK, or name
- Search and filter SEC filings by form type and date
- Parse filings into structured Python objects (10-K, 10-Q, 8-K)
- Extract financial statements and key metrics
- Track insider trading from Form 4
- Analyze institutional holdings from 13F filings

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Get Any Company's Financial Data in 3 Lines

Look up a company by ticker and pull its latest financials -- no API key, no query syntax:

In [ ]:
company = Company("AAPL")
financials = company.get_financials()

print(f"Company: {company.name}")
print(f"Revenue: ${financials.get_revenue():,.0f}")
print(f"Net Income: ${financials.get_net_income():,.0f}")

## Look Up Any Company

The `Company()` constructor accepts ticker symbols or CIK numbers:

In [ ]:
# By ticker
apple = Company("AAPL")
print(f"{apple.name:30s} CIK: {apple.cik:>10}  SIC: {apple.sic}")

# By CIK number
msft = Company(789019)
print(f"{msft.name:30s} CIK: {msft.cik:>10}  SIC: {msft.sic}")

# Another ticker
nvda = Company("NVDA")
print(f"{nvda.name:30s} CIK: {nvda.cik:>10}  SIC: {nvda.sic}")

## Search a Company's Filings

Every company's full filing history is available. Filter by form type to find exactly what you need:

In [ ]:
company = Company("AAPL")

# All filings
all_filings = company.get_filings()
print(f"Total filings: {len(all_filings)}")

# Filter by form type
annual_reports = company.get_filings(form="10-K")
quarterly_reports = company.get_filings(form="10-Q")
current_reports = company.get_filings(form="8-K")

print(f"10-K annual reports:   {len(annual_reports)}")
print(f"10-Q quarterly reports: {len(quarterly_reports)}")
print(f"8-K current reports:   {len(current_reports)}")

# Show the most recent 10-K
annual_reports.head(3)

## Search Filings Across All Companies

Use `get_filings()` to search across the entire SEC EDGAR database by date, form type, year, or quarter:

In [ ]:
# All filings on a specific date
filings = get_filings(filing_date="2026-02-10")
print(f"All filings on 2026-02-10: {len(filings):,}")

# Filter by form type and date range
tenks = get_filings(filing_date="2025-12-01:2025-12-05", form="10-K")
print(f"10-K filings Dec 1-5, 2025: {len(tenks)}")

# Search by quarter
q4 = get_filings(year=2025, quarter=4, form="10-K")
print(f"10-K filings in Q4 2025: {len(q4)}")

## Parse Any Filing into a Python Object

Call `.obj()` on any filing to get a structured Python object with typed properties and methods:

In [ ]:
# Parse a 10-K into a TenK object
filing = Company("MSFT").get_filings(form="10-K")[0]
tenk = filing.obj()

print(f"Type: {type(tenk).__name__}")
print(f"\nBusiness description (first 300 chars):")
print(tenk.business[:300])

## Extract Financial Data

Get key financial metrics from any company's latest filing with `get_financials()`:

In [ ]:
import pandas as pd

tickers = ["AAPL", "MSFT", "NVDA", "GOOG"]
rows = []

for ticker in tickers:
    fin = Company(ticker).get_financials()
    rows.append({
        "Ticker": ticker,
        "Revenue ($B)": fin.get_revenue() / 1e9,
        "Net Income ($B)": fin.get_net_income() / 1e9,
        "Total Assets ($B)": fin.get_total_assets() / 1e9,
    })

pd.DataFrame(rows).set_index("Ticker").round(1)

## Get Multi-Year Financial History

Use `get_facts()` for historical data -- revenue, earnings, and other metrics going back years:

In [ ]:
facts = Company("AAPL").get_facts()

# Revenue time series
revenue = facts.time_series("Revenues")
print(f"Revenue history: {len(revenue)} periods")

# Multi-year income statement
facts.income_statement()

## Track Insider Trading (Form 4)

SEC Form 4 filings disclose when company officers and directors buy or sell stock:

In [ ]:
company = Company("AAPL")
form4_filings = company.get_filings(form="4")

# Find a filing with actual trades
for f in form4_filings.head(10):
    f4 = f.obj()
    if f4.shares_traded > 0:
        print(f"Insider:  {f4.insider_name}")
        print(f"Position: {f4.position}")
        print(f"Date:     {f4.reporting_period}")
        print(f"Shares:   {f4.shares_traded:,}")
        print(f"\nTrades:")
        print(f4.market_trades[["Security", "Date", "Shares", "Price", "AcquiredDisposed"]])
        break

## Analyze Institutional Holdings (13F)

13F filings reveal what stocks institutional investors hold:

In [ ]:
# Berkshire Hathaway's portfolio
brk = Company("BRK-B")
thirteenf = brk.get_filings(form="13F-HR")[0].obj()

print(f"Manager: {thirteenf.management_company_name}")
print(f"Holdings: {thirteenf.total_holdings:,}")
print(f"Value: ${thirteenf.total_value:,.0f}")

# Top 5 holdings by value
thirteenf.holdings.nlargest(5, "Value")

## Extract Filing Text, Markdown, or HTML

Every filing can be converted to text, markdown, or HTML for NLP, search, or LLM context:

In [ ]:
filing = Company("NVDA").get_filings(form="10-K")[0]

text = filing.text()
markdown = filing.markdown()
html = filing.html()

print(f"Text:     {len(text):>10,} chars")
print(f"Markdown: {len(markdown):>10,} chars")
print(f"HTML:     {len(html):>10,} chars")
print(f"\nFirst 300 chars of text:")
print(text[:300])

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing SEC EDGAR data:

**With edgartools (free, no API key):**
```python
company = Company("AAPL")
filing = company.get_filings(form="10-K")[0]
tenk = filing.obj()             # Structured Python object
fin = company.get_financials()  # Latest financial metrics
text = filing.text()            # Full filing text
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import QueryApi, XbrlApi, RenderApi
query_api = QueryApi(api_key="YOUR_PAID_API_KEY")
xbrl_api = XbrlApi(api_key="YOUR_PAID_API_KEY")
# ... build query syntax, paginate results, parse JSON manually
```

With edgartools, SEC data is accessible through simple Python objects -- no query syntax, no API key, no monthly fee.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Company lookup ──
company = Company("AAPL")           # By ticker
company = Company(320193)           # By CIK

# ── Filings ──
company.get_filings(form="10-K")    # Company filings by form
get_filings(filing_date="2026-02-10")  # All filings by date
get_filings(year=2025, quarter=4, form="10-K")  # By quarter

# ── Parse filings ──
filing.obj()                        # TenK, TenQ, EightK, Form4, etc.
filing.text()                       # Plain text
filing.markdown()                   # Markdown
filing.html()                       # HTML

# ── Financial data ──
company.get_financials()            # Latest metrics
company.get_facts()                 # Historical data
facts.income_statement()            # Multi-year income statement
facts.time_series("Revenues")       # Revenue history

# ── Insider trading & holdings ──
company.get_filings(form="4")       # Form 4 insider trades
company.get_filings(form="13F-HR")  # 13F institutional holdings
```

## What's Next

You've learned the core SEC EDGAR API in Python. Dive deeper with these tutorials:

- [Download and Parse 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-10k-annual-report-python.ipynb)
- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)
- [Track Insider Trading from SEC Form 4](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/insider-trading-sec-form4-python.ipynb)
- [Analyze 13F Institutional Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/13f-institutional-holdings-python.ipynb)
- [Compare Company Financials](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/compare-company-financials-python.ipynb)
- [Download SEC Filings in Bulk](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-sec-filings-bulk-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*